# SPRINT 8. Parte II
> PowerBI + Python

## Nivel 1
Pasar todos los gráficos del S8_1N1 a PowerBI.

El Script de conexión e importación de datos es este:

In [ ]:
import dateutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import configparser
import mysql.connector

cfgFp = open(r'C:\Users\formacio\Downloads\mlg\git_repo\SPRINT8\db.ini', mode='r')
iniFp = configparser.ConfigParser()
iniFp.read_file(cfgFp)
dbCfg = dict(iniFp.items('database'))

# exit()

db = mysql.connector.connect(
    host=dbCfg['host'],
    database=dbCfg['database'],
    user=dbCfg['user'],
    password=dbCfg['password']
)

tr_df   = pd.read_sql("SELECT * FROM `transaction`", db)

user_df = pd.read_sql("SELECT * FROM `user`", db)

co_df = pd.read_sql("SELECT * FROM company", db)

pr_df   = pd.read_sql("SELECT * FROM product", db)
prtr_df = pd.read_sql("SELECT * FROM product_transaction", db)

def age(birthdate):
    dob = pd.to_datetime(birthdate).date() if type(birthdate) is not str \
                  else dateutil.parser.parse(birthdate).date()
    hoy = dateutil.utils.today().date()
    ddiff = hoy - dob
    edad  = int(ddiff.days / 365.25)
    return edad

# Add age column to user dataframe
user_df["age"] = user_df['birthdate'].map(age)


### S8.2N1E1
Distribución de edades de los usuarios. El script utilizado es:

In [ ]:
import matplotlib.pyplot as plt

user_df = dataset
user_df.groupby('age')['age'] \
        .count() \
        .plot.bar(
            x='age',
            # figsize=(10,5),
            title="Age distribution",
            ylabel="# of Users",
            xlabel="Age"
        )
plt.show()

In [ ]:
# Y el histograma es este. Aún no estoy seguro si dejarlo.


### S8.1N1E2
Dos variables numéricas.
El código adaptado ha quedado así:

In [ ]:
from matplotlib import pyplot, ticker
import numpy as np

productos = dataset
salesXage = productos.get(['age', 'amount']) \
                     .groupby('age')['amount'] \
                     .agg(['sum', 'count'])

f, sxa_gr = pyplot.subplots(figsize=(12,8))
ages_list = salesXage.index
bar_width = 0.4
positions = np.arange(len(ages_list))

sxa_gr.bar(
    x=positions,
    height=salesXage['count'], # transactions by age
    width=bar_width,
    label='# of sales',
    color='#5c5'
)

sxa_gr.set_xlabel('Age')
sxa_gr.set_ylabel('# of transactions')
sxa_gr.set_xticks(positions, [str(n) for n in ages_list])

sxa_gr2 = sxa_gr.twinx()
sxa_gr2.bar(
    x=positions+bar_width,
    height=salesXage['sum'] / 1000,
    width=bar_width,
    label='Sales in M€',
    color='#393'
)
sxa_gr2.set_ylabel('Sales in (in thousands of €)')
sxa_gr2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%dM €'))

sxa_gr.set_title("Sales and revenue by customer age")
sxa_gr.legend(loc='upper left')
sxa_gr2.legend(loc='upper right')

pyplot.show()